# Matcher

In [ ]:
#| default_exp matcher
%load_ext autoreload
%autoreload 2

### Overview
In the previous module, the LHS Parser received an LHS string, describing some graph pattern, and parsed it to an equivalent NetworkX DiGraph. In the following module, we search for **matches** to this pattern in our input graph - That is, find all subgraphs of our input graph, which have the same structure as the pattern in terms of nodes, the edges connecting them and the attributes they all have. 

Each match is basically a mapping from a subset of the input graph nodes to a the pattern nodes (as we can deduce what are the matched edges and attributes accordingly).

The **Matcher** in this module does the following:
* **Searches for matches** in the input graph, according to some LHS pattern graph.
* **Filters matches** based on either an explicit boolean function, or constraints given to the matcher by the parser to be handled later.
* **Constructs a list of Match objects**, each corresponds to one of the filtered matches we've found. The list will be used in later modules, and will also allow users to see the matches and to use them imperatively.

The final, filtered list of Match objects is returned from this module's main function, **find_matches**.

### Requirements

In [ ]:
#| export
from networkx import DiGraph
from networkx.algorithms import isomorphism # check subgraph's isom.
import itertools # iterating over all nodes\edges combinations
from graph_rewrite.match_class import Match, mapping_to_match
from graph_rewrite.core import NodeName, _create_graph, _plot_graph
from typing import *

### Find Matches
Given an input graph and a pattern graph, we want to find the list of matches from pattern nodes to input-graph nodes, each constructs a corresponding Match object. 

#### Checking for Attributes Existance
A significant part of finding matches is comparing the attributes of different nodes to each other, in order to know whether can match. The Parser, which constructs the pattern graph, does not include explicit required values of attributes in that graph (it saves these values somewhere else, we'll get to that later).

Therefore, given the pattern graph, we can only tell whether some pattern node has the same attributes (regardless of their values) as some input graph node. This is done by the following function:

In [ ]:
#| export
def _attributes_exist(input_graph_attrs: dict, pattern_attrs: dict) -> bool:
    """Given an input-graph node and a pattern node, checks whether the input-graph node
    has all the attributes which the pattern node requires. If it does, then this input-graph node might be matched as that pattern node.
    
    Note that we only refer to the existence of attributes here, i.e. if both nodes have the same attributes but with
    different values, this function still considers them as a potential pair to match.

    Args:
        input_graph_attrs (dict): Attributes of some input-graph node.
        pattern_attrs (dict): Attributes of some pattern-graph node.

    Returns:
        bool: True if the input-graph node has all the attributes which the pattern node requires, False otherwise.
    """
    return set(pattern_attrs.keys()).issubset(set(input_graph_attrs.keys()))

#### Finding Structural Matches
NetworkX provides an out-of-the-box isomorphism matcher, which compares the structure of two graphs and tells whether they are isomorphic (have the same nodes and edges). We utilize this isomorphism matcher by beginning our matching process with structural matches only (ignoring the attributes totally):

In [ ]:
#| export
def _find_structural_matches(graph: DiGraph, pattern: DiGraph) -> list[Tuple[DiGraph, dict[NodeName, NodeName]]]:
    """Given a graph, find all of its subgraphs which have the same structure (same nodes and edges)
    as a given pattern DiGraph. That is, all subgraphs which are isomorphic to the pattern.

    Args:
        graph (DiGraph): A graph to find matches in
        pattern (DiGraph): A pattern graph produced by the LHS Parser.

    Returns:
        list[Tuple[DiGraph, dict[NodeName, NodeName]]]: list of (subgraph, mapping) tuples,
            where for each pair, the subgraph is the subset of nodes and edges in the input graph that
            match the pattern, and the mapping is a dictionary that maps nodes in that subgraph
            to nodes in the pattern.
    """

    isom_matches = []
    for sub_nodes in itertools.combinations(graph.nodes, len(pattern.nodes)):
        nodes_subg: DiGraph = graph.subgraph(sub_nodes)
        for sub_edges in itertools.combinations(nodes_subg.edges(data=True), len(pattern.edges)):
            # Create a subgraph with selected edges and nodes
            subg = DiGraph()
            subg.add_nodes_from(list(nodes_subg.nodes(data=True)))
            subg.add_edges_from(list(sub_edges))

            # Find structural matches with the selected edges and nodes
            matcher = isomorphism.DiGraphMatcher(pattern, subg)
            for isom_mapping in matcher.isomorphisms_iter():
                isom_matches.append((nodes_subg, isom_mapping))
                
    return isom_matches

#### Expanding Matches With Attribtues
Now that we have all structural matches between the graph and the pattern, we want to take our attributes into account.
Currently, we still ignore the values of the attributes, and check whether some structural match also has the attributes of the pattern in the matched subgraph:

In [ ]:
#| export
def _isom_matches_pattern(isom: Tuple[DiGraph, dict], pattern: DiGraph) -> bool:
    """Given a graph that is isomorphic to the pattern, checks whether they also
    match in terms of their attributes (that is, the graph has the same attributes
    as the pattern). If they does, then the isomorphic graph matches the pattern completely.

    Args:
        isom (Tuple[DiGraph, dict]): A graph that's isomorphic to the pattern
        pattern (DiGraph): A pattern graph produced by the LHS Parser.

    Returns:
        bool: True if the isomorphic graph matches the pattern, False otherwise.
    """

    # check nodes match
    subgraph, mapping = isom
    if not all([_attributes_exist(subgraph.nodes[original_node], pattern.nodes[pattern_node]) \
                for (pattern_node, original_node) in mapping.items()]):
        return False

    # check edges match
    if all([_attributes_exist(subgraph.edges[mapping[edge[0]], mapping[edge[1]]], edge[2]) \
                for edge in pattern.edges(data=True)]):
        return True

#### Filtering Matches
The only thing we ignored up until now is attribute values. As metioned above, the LHS Parser does not include required attribute values in the pattern graph. Instead, it constructs a boolean function which receives a Match object and checks whether the match it represents has the required attribute values (if there are any). 

This boolean function is further extended by the user of the library, which can pass as parameter a function of the same format, which filteres a list of Match objects based on any condition it wishes to apply. The LHS Parser, in addition to the pattern graph, provides the extended filtering function, that mixes both the user and the parser constraints which were not handled by the matcher so far.

Later in this module, we will use the extended function to filter the list of Match objects we get from the structural and attribute-existence-based matchers. The signature of that function will be as follows:

In [ ]:
#| export
FilterFunc = Callable[[Match], bool]

#### Putting It All Togehter
Given our ability to find matches (both structural and in terms of attribute existence) between two graphs, as well as filtering matches according to desired conditions and constraints, we can finally find complete matches of the pattern in our input graph. 

Using the functions presented thus far, the search for matches might take a lot of time if the graph has a high number of nodes / edges. Nodes which are no real candidate to match any pattern node (do not share attributes with any pattern node) are checked eitherway, which is extermely inefficient. 

Therefore, before we search for matches in our input graph, we will reduce it to only contain the nodes that might match any of the pattern nodes (and their connected edges as well). This might improve the whole matching performance. The following function is used in order to do exactly that:

In [ ]:
#| export
def _node_matches_pattern(graph_node_attrs: dict, pattern: DiGraph) -> bool:
    """Given the attributes of some input-graph node, checks whether this node
    has the same attributes as some node in the pattern graph. If it does,
    then the node by match that node in the pattern - and thus, might be included
    in a subgraph that will match the entire pattern.

    Args:
        graph_node_attrs (dict): Attributes of some input-graph node.
        pattern (DiGraph): A pattern graph produced by the LHS Parser.

    Returns:
        bool: True if the input-graph node has the same attributes as some pattern node, False otherwise.
    """
    return any([_attributes_exist(graph_node_attrs, pattern_attr) for (_, pattern_attr) in pattern.nodes(data=True)])

We are now combining everything we saw in order to find the matches of a pattern in our input graph. The matches are returned as a (filtered) list of Match objects:

In [ ]:
#| export
def find_matches(input_graph: DiGraph, pattern: DiGraph, condition: FilterFunc = lambda match: True) -> List[Match]:
    """Find all matches of a pattern graph in an input graph, for which a certain condition holds.
    That is, subgraphs of the input graph which have the same nodes, edges, attributes and required attribute values
    as the pattern defines, which satisfy any additional condition the user defined.

    Args:
        input_graph (DiGraph): A graph to find matches in
        pattern (DiGraph): A pattern graph produced by the LHS Parser.
        condition (FilterFunc, optional): A function which recives a Match objects, and checks whether some condition holds
            for the corresponding match. Defaults to a condition function which always returns True.

    Returns:
        List[Match]: List of Match objects, each corresponds to a match of the pattern in the input graph.
    """

    # Narrow down search space by keeping only input-graph nodes that have the same attributes as some pattern node
    matching_nodes = [n for (n, attrs) in input_graph.nodes(data=True) if _node_matches_pattern(attrs, pattern)]
    # Reducing the input graph to the matching nodes + connected edges
    reduced_input_g = input_graph.subgraph(matching_nodes)
    
    # Find all structural matches (isomorphisms), ignore attributes
    isom_matches =  _find_structural_matches(reduced_input_g, pattern)
    # Find matches with attributes among isoms (match pattern's attributes)
    attribute_matches = [mapping for (subgraph, mapping) in isom_matches if _isom_matches_pattern((subgraph, mapping), pattern)]

    # construct a list of Match objects
    matches_list = [mapping_to_match(input_graph, pattern, mapping) for mapping in attribute_matches]
    return [match for match in matches_list if condition(match)]

### Tests

#### Test Utils

In [ ]:
def _test_match(input, pattern, expected):
    matches = find_matches(input, pattern)
    return all([match.mapping in expected for match in matches])

#### Test Cases

In [ ]:
# Simple graph tests
input_graph = _create_graph(
    ['A','B','C','D'], 
    [
        ('A','B'),
        ('A','C'),
        ('A', 'A'),
        ('C', 'C'),
        ('A', 'C')
    ]
)

pattern1 = _create_graph(['X'], [('X', 'X')])
assert _test_match(input_graph, pattern1, [{'X': 'A'}, {'X': 'C'}])
assert _test_match(input_graph, pattern1, [{'X': 'C'}, {'X': 'A'}])
assert not _test_match(input_graph, pattern1, [{'X': 'A'}])

pattern2 = _create_graph(['X', 'Y'], [('X', 'X')])
assert _test_match(input_graph, pattern2, [{'X': 'A', 'Y': 'B'}, {'X': 'A', 'Y': 'C'}, {'X': 'A', 'Y': 'D'}, 
                                    {'X': 'C', 'Y': 'A'}, {'X': 'C', 'Y': 'B'}, {'X': 'C', 'Y': 'D'}])

pattern3 = _create_graph(['X', 'Y'], [('X', 'X'), ('X', 'Y')])
assert _test_match(input_graph, pattern3, [{'X': 'A', 'Y': 'B'}, {'X': 'A', 'Y': 'C'}])

pattern4 = _create_graph(['X', 'Y'], [('X', 'X'), ('Y', 'X')])
assert _test_match(input_graph, pattern4, [{'X': 'C', 'Y': 'A'}])

pattern5 = _create_graph(['X', 'Y'], [('X', 'X'), ('Y', 'X'), ('X', 'Y')])
assert _test_match(input_graph, pattern5, [])

pattern6 = _create_graph(['1','2','3','4','5'], [])
assert _test_match(input_graph, pattern6, [])

input_graph.add_edge('C', 'D')
input_graph.add_edge('D', 'A')
pattern7 = _create_graph(['X', 'Y', 'Z'], [('X', 'Y'), ('Y', 'Z')])
assert _test_match(input_graph, pattern7, [{'X': 'A', 'Y': 'C', 'Z': 'D'},
                                    {'X': 'C', 'Y': 'D', 'Z': 'A'},
                                    {'X': 'D', 'Y': 'A', 'Z': 'C'},
                                    {'X': 'D', 'Y': 'A', 'Z': 'B'}])


In [ ]:
# POC: High number of nodes, solved with attribute filtering
num_nodes = 100000

input_graph = _create_graph(
    [n for n in range(num_nodes)] + [(num_nodes+1, {'attr': 15}), (num_nodes+2, {'attr': 15})], 
    [
        (num_nodes+1, num_nodes+2),
        (2,4),
        (3,1)
    ]
)

pattern = _create_graph([('X', {'attr': None}), ('Y', {'attr': None})], [('X', 'Y')])
assert _test_match(input_graph, pattern, [{'X': num_nodes+1, 'Y': num_nodes+2}])